In [1]:
import asyncio

class CognitiveFeedbackRouter:
    def __init__(self):
        self.number = 0
        self.user_input = ''
        self.count_completed = asyncio.Event()
        self.input_received = asyncio.Event()
        self.input_processed = asyncio.Event()

    async def contemplation(self):
        for self.number in range(1, 6):
            print(f"Current number: {self.number}")
            await asyncio.sleep(1)
        if self.input_received.is_set():
            self.count_completed.set()  # Signal to process input
            await self.input_processed.wait()  # Wait for input processing to complete
            self.input_processed.clear()
        self.input_received.clear()  # Ready for new input

    async def process_input(self):
        print(f"Processing user input: '{self.user_input}'")
        await asyncio.sleep(3)  # Simulate processing time
        self.input_processed.set()  # Signal that input processing is complete

    async def get_user_input(self):
        while True:
            self.user_input = await asyncio.get_event_loop().run_in_executor(None, input, "Enter something: ")
            self.input_received.set()  # Signal that input has been received
            await self.count_completed.wait()  # Wait for the count to complete before processing
            self.count_completed.clear()  # Reset for the next round
            await self.process_input()  # Process the input

    async def run(self):
        input_task = asyncio.create_task(self.get_user_input())

        while True:
            await self.contemplation()  # Restart contemplation after each completion

# Apply the necessary setup for Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

# Create an instance of the class and run it
router = CognitiveFeedbackRouter()
asyncio.run(router.run())

Current number: 1
Current number: 2
Current number: 3
Current number: 4
Current number: 5
Current number: 1


Enter something:  a


Current number: 2
Current number: 3
Current number: 4
Current number: 5
Processing user input: 'a'
Current number: 1
Current number: 2


KeyboardInterrupt: 